# Imports

In [ ]:
import math
import time
import copy
import tqdm
import glob
import os
from IPython.display import Audio
import numpy as np
import pandas as pd
# import itables
# itables.init_notebook_mode()
from sklearn.model_selection import ParameterGrid
import librosa

In [ ]:
# from acestep.ui.components import create_main_demo_ui
from acestep.pipeline_ace_step import ACEStepPipeline
# from acestep.data_sampler import DataSampler

In [ ]:
# # !ls outputs
# display(Audio(ASH_TEST_AUDIO_PATH))
# Audio(output_paths)#, autoplay=True)

# Initialization

In [ ]:
# checkpoint_path="~/.cache/ace-step/checkpoints/models--ACE-Step--ACE-Step-v1-3.5B/"
# checkpoint_path="f"

# LORA_DIR='/home/rileywilliams/soundverse/ACE-step-lowvram/models/exp/ace_step_lora_rjwpersonal/'
# LORA_PATH=os.path.join(LORA_DIR, "epoch=56-step=6000_lora")
# LORA_PATH='none'
# LORA_PATH="/home/rileywilliams/soundverse/ACE-step-lowvram/ace_step_lora/shoujobyou"
BF16=False
TORCH_COMPILE=True
CPU_OFFLOAD=True
OVERLAPPED_DECODE=True # maybe try false?

try:
    del model_demo
except:
    pass
model_demo = ACEStepPipeline(
    # checkpoint_dir=checkpoint_path,
    dtype="bfloat16" if BF16 else "float32",
    torch_compile=TORCH_COMPILE,
    cpu_offload=CPU_OFFLOAD,
    overlapped_decode=OVERLAPPED_DECODE
)

Entropy Rectifying Guidance (ERG) for tag-transformer/lyrics/diffusion model. It will multiple a temperature to the attention to make a weaker tag condition and make better diversity.

In [ ]:
# 1 / math.sqrt(64)
# # model_demo.ace_step_transformer.unload_lora()
# wget https://huggingface.co/woctordho/ACE-Step-v1-LoRA-collection/resolve/main/ace_step_v1_lora_shoujobyou.safetensors?download=true

In [ ]:
# gcloud compute scp --recurse --compress --zone "us-west1-b" --project "theapp-388314" \
# "/Users/rileywilliams/Music/Logic/SoundVerse/avi_ash_PROCESSED" \
# rileywilliams@dev-gpu-rjw:"/mnt/disks/audio-ai-research-speech-data"

# Generation

### With LoRa

In [ ]:
LYRICS='[Instrumental]'
# LYRICS=''
# LORA_WEIGHT=1.0
LORA_WEIGHT=1 / math.sqrt(64) # where 64 is the rank of the LoRa
# LORA_WEIGHT=1 / 64
# LORA_WEIGHT=0.05 # 1.0
LORA_TRIGGER_WORD='4abe0dc877704be489c89dc07927921f'
PROMPT=f"{LORA_TRIGGER_WORD}. calm electronic melodic house at sunset"
# PROMPT="calm electronic melodic house at sunset"
# PROMPT=f'{LORA_TRIGGER_WORD}'
# PROMPT=''
MANUAL_SEEDS = 42
AUDIO_DURATION = 30

In [ ]:
output_paths, input_params_json = model_demo(
    # format: str = "wav",
        audio_duration=AUDIO_DURATION, #: float = 60.0,
        prompt=PROMPT,
        lyrics=LYRICS, #     lyrics: str = None,
    #     infer_step: int = 60,
    #     guidance_scale: float = 15.0,
    #     scheduler_type: str = "euler",
        cfg_type = "apg",
        omega_scale = 20.0, #(this is "granularity" in the GUI, defaults to 10)
    #     omega_scale: int = 10.0, (this is "granularity" in the GUI)
        manual_seeds=MANUAL_SEEDS, # None,
        guidance_interval=1.0, #     guidance_interval: float = 0.5,
    #     guidance_interval_decay: float = 0.0,
    #     min_guidance_scale: float = 3.0,
    #     use_erg_tag: bool = True,
    #     use_erg_lyric: bool = True,
    #     use_erg_diffusion: bool = True,
    #     oss_steps: str = None,
        guidance_scale_text = 5.0,
        guidance_scale_lyric = 1.5,
    #     guidance_scale_text: float = 0.0, # 5.0
    #     guidance_scale_lyric: float = 0.0, # 1.5
    #     audio2audio_enable: bool = False,
    #     ref_audio_strength: float = 0.5,
    #     ref_audio_input: str = None,
        # lora_name_or_path='none',
        lora_name_or_path=LORA_PATH, # "none
        lora_weight=LORA_WEIGHT,
        # retake_seeds: list = None,
        # retake_variance: float = 0.5,
        # task: str = "text2music",
        # src_audio_path: str = None,
        # edit_target_prompt: str = None,
        # edit_target_lyrics: str = None,
        # edit_n_min: float = 0.0,
        # edit_n_max: float = 1.0,
        # edit_n_avg: int = 1,
        # save_path: str = None,
        # batch_size: int = 1,
        # debug: bool = False,
)
# data_sampler = DataSampler()

# demo = create_main_demo_ui(
#     text2music_process_func=model_demo.__call__,
#     sample_data_func=data_sampler.sample,
#     load_data_func=data_sampler.load_json,
# )

### Notes

In [ ]:
# Using Riley's LoRa, with lyrics as empty string
# 'calm electronic melodic house at sunset',
# './outputs/output_20250617003852_0.wav'

# Using Riley's LoRa, with lyrics='[Instrumental]'
# 'calm electronic melodic house at sunset',
# './outputs/output_20250617005807_0.wav'

# Using Riley's LoRa, with lyrics='[Instrumental]', and f"{LORA_TRIGGER_WORD} calm electronic melodic house at sunset"
# 'calm electronic melodic house at sunset',
# './outputs/output_20250617010151_0.wav'

# Using Riley's LoRa, with lyrics='[Instrumental]', and
# PROMPT='4abe0dc877704be489c89dc07927921f,Dance music, ambient music,In the genre of electronic and dance and ambient,118 bpm'
# './outputs/output_20250617011301_0.wav'

# Using Riley's LoRa, with lyrics='[Instrumental]', and
# PROMPT='4abe0dc877704be489c89dc07927921f'
# './outputs/output_20250617011503_0.wav'

# Using Riley's LoRa, with lyrics='[Instrumental]', and
# PROMPT=''
# './outputs/output_20250617012344_0.wav'

# Using Riley's LoRa, with lyrics='', and PROMPT=''
# './outputs/output_20250617012759_0.wav'

# ===== ===== ===== ===== =====

# Using WITHOUT Riley's LoRa
# 'calm electronic melodic house at sunset',
# './outputs/output_20250617005206_0.wav'
# with lyrics as empty string, I get chinese singing :/

# Using WITHOUT Riley's LoRa, but with lyrics='[Instrumental]' (now it works fine!)
# 'calm electronic melodic house at sunset',
# './outputs/output_20250617005541_0.wav'
# with lyrics as empty string, I get chinese singing :/


# , input_params_json

### Output

In [ ]:
# Using Riley's LoRa, with lyrics='', and PROMPT=''
# './outputs/output_20250617012759_0.wav'
# Most recent (riley's lora): './outputs/output_20250617155406_0.wav'
output_paths

In [ ]:
print('rileys')
display(Audio('./outputs/output_20250617155406_0.wav', autoplay=False))
print('base')
display(Audio('./outputs/output_20250617160348_0.wav', autoplay=False))
print('bad')
Audio(output_paths, autoplay=False)

In [ ]:
# LYRICS='[Instrumental]'
# # LYRICS=''
# LORA_WEIGHT=0.2 # 1.0
# LORA_TRIGGER_WORD='4abe0dc877704be489c89dc07927921f'
# PROMPT=f"{LORA_TRIGGER_WORD} calm electronic melodic house at sunset"
# './outputs/output_20250617013545_0.wav'

# LYRICS='[Instrumental]'
# # LYRICS=''
# LORA_WEIGHT=0.05 # 1.0
# LORA_TRIGGER_WORD='4abe0dc877704be489c89dc07927921f'
# PROMPT=f"{LORA_TRIGGER_WORD} calm electronic melodic house at sunset"
# './outputs/output_20250617014204_0.wav'

In [ ]:
# model_demo.__call__

### Using Audio2Audio

In [ ]:
ASH_AUDIO_PATH = "/mnt/disks/audio-ai-research-speech-data/avi_ash_PROCESSED"
all_ash_paths = glob.glob(os.path.join(ASH_AUDIO_PATH, '*'))
# ASH_TEST_AUDIO_PATH = os.path.join(ASH_AUDIO_PATH, "15_EDOM__005.wav") # This one is just too chaotic?
ASH_TEST_AUDIO_PATH = os.path.join(ASH_AUDIO_PATH, "12_Always On The Run__003.wav")

In [ ]:
# file_index = -36
# ASH_TEST_AUDIO_PATH = all_ash_paths[file_index]
# display(Audio(ASH_TEST_AUDIO_PATH, autoplay=True))

In [ ]:
# input_params_json

In [ ]:
MANUAL_SEEDS=296772417648513
LYRICS='[Instrumental]'
PROMPT='lofi beats'
# PROMPT='classical, piano, instrumental'
# PROMPT='soft jazz, light piano'
# PROMPT="edm, synth, bass, euphoric, energetic"
# PROMPT=""
# PROMPT="hardcore metal riffs"
AUDIO_DURATION = 43
output_paths, input_params_json = model_demo(
    # format: str = "wav",
        audio_duration=AUDIO_DURATION, #: float = 60.0,
        prompt=PROMPT,
        lyrics=LYRICS, #     lyrics: str = None,
        infer_step = 100,

        # guidance_scale = 15.0, # When guidance_scale_lyric > 1 and guidance_scale_text > 1, the guidance scale will not be applied.
        scheduler_type = "heun",
        # scheduler_type = "pingpong",
        # scheduler_type = "euler", # "euler" is recommended, heun will take more time
        cfg_type = "apg",
        # cfg_type = "cfg",
        omega_scale = 10.0, # (this is "granularity" in the GUI, defaults to 10)
        # omega_scale: int = 10.0,
        manual_seeds=MANUAL_SEEDS, # None,
        guidance_scale=15,
        guidance_interval=0.5,
        # guidance_interval=1.0,
        # guidance_interval: float = 1.0,
        guidance_interval_decay = 0.0,
        min_guidance_scale = 3.0,
    
        # use_erg_tag=False,
        # use_erg_lyric=False,
        # use_erg_diffusion=False,
        use_erg_tag=True,
        use_erg_lyric=True,
        use_erg_diffusion=True,
    
    #     use_erg_tag: bool = True,
    #     use_erg_lyric: bool = True,
    #     use_erg_diffusion: bool = True,
    #     oss_steps: str = None,
    #     guidance_scale_text: float = 0.0,
    #     guidance_scale_lyric: float = 0.0,
        # guidance_scale_text = 7,
        # guidance_scale_lyric = 2,
    
        # audio2audio_enable = True,
        # ref_audio_strength = 0.10, # 0.18 was good on Gradio site...
        # ref_audio_input = ASH_TEST_AUDIO_PATH,
)
print(output_paths)

In [ ]:
# !ls outputs
print(PROMPT)
# display(Audio(ASH_TEST_AUDIO_PATH))
Audio(output_paths)#, autoplay=True)

In [ ]:
input_params_json

# Set up Hyperparam Search

## Generate Parameter Grid

In [ ]:
def merge_params(target, recorded):
    merged_params = copy.copy(recorded)
    for _key, _value in target.items():
        merged_params[_key] = _value
    return merged_params

In [ ]:
MANUAL_SEEDS=296772417648513
AUDIO_DURATION = 43

prompts_list = [
    'lofi beats',
    # 'soft jazz with light piano',
    "edm, synth, bass, euphoric"
]

lyrics_list = [
    '[Instrumental]'
]

track_list = [
    ASH_TEST_AUDIO_PATH, None
]

param_grid_dict = {
    "audio_duration" : [AUDIO_DURATION],
    'prompt': prompts_list,
    'lyrics': lyrics_list,
    'ref_audio_input': track_list,
    'guidance_scale': [30, 15], # Max guidance before decay (30 is highest)
    'scheduler_type': ['heun', 'pingpong'],
    'infer_step': [70], 
    # 'ref_audio_strength': np.arange(.1, .36, .1),
    'ref_audio_strength':[.075, .1, .15],
    'use_erg': [False, True], # I think if erg is used, then cfg/apg isn't used?
    'manual_seeds': [MANUAL_SEEDS],
    'omega_scale': [100, 10], # Granularity, 100 is max
    'guidance_interval': [0.5, 0.9, 1.0],
    "guidance_interval_decay": [0.0, 0.25, 0.75],
    "min_guidance_scale": [3],
}

param_grid = ParameterGrid(param_grid_dict)
len(param_grid)
# param_grid_dict

### Prepare param grid dataframe

In [ ]:
experiment_name='exp_03'
audio_outputs_path = f'/mnt/disks/ace_step_generation/{experiment_name}'
os.makedirs(audio_outputs_path, exist_ok=True)

grid_search_input_dataframe_filepath = os.path.join(audio_outputs_path, 'grid_search_input_data.csv')

# if starting from scratch
grid_dataframe = pd.DataFrame(param_grid)
grid_dataframe['complete'] = False
grid_dataframe['audio_outputs_path'] = audio_outputs_path
# If restarting from file
# grid_dataframe = pd.read_csv(grid_search_input_dataframe_filepath)


grid_dataframe.to_csv(
    path_or_buf=grid_search_input_dataframe_filepath,
    mode='w',
    index=False
)
grid_search_output_dataframe_filepath = os.path.join(audio_outputs_path, 'grid_search_output_data.csv')

#### Filter audio strength options for non audio2audio

In [ ]:
print(f'starting with {grid_dataframe.shape[0]} variations')
default_ref_audio = 0.1

repeat_condition = grid_dataframe['ref_audio_input'].isna()

grid_dataframe.loc[
    repeat_condition,
    'ref_audio_strength'
]= default_ref_audio
grid_dataframe = grid_dataframe.drop_duplicates().reset_index(drop=True)
print(f'ending with {grid_dataframe.shape[0]} variations')

In [ ]:
# grid_dataframe = grid_dataframe.sort_values(
#     by=['ref_audio_input', 'prompt','infer_step', 'guidance_scale', 'use_erg', 'scheduler_type'], 
#     ascending=[True, True, False, False, True, True]
# )
# grid_dataframe

In [ ]:
def generate_song(params):
    output_paths, input_params_json = model_demo(
        audio_duration=params['audio_duration'],
        prompt=params['prompt'],
        lyrics=params['lyrics'],
        infer_step = params['infer_step'],
        scheduler_type = params['scheduler_type'],
        manual_seeds=params['manual_seeds'],
        guidance_interval=params['guidance_interval'],
        guidance_scale=params['guidance_scale'],
        guidance_interval_decay=params['guidance_interval_decay'],
        min_guidance_scale=params['min_guidance_scale'],
        use_erg_tag=params['use_erg_tag'],
        use_erg_lyric=params['use_erg_lyric'],
        use_erg_diffusion=params['use_erg_diffusion'],
        omega_scale=params['omega_scale'],
        audio2audio_enable = params['audio2audio_enable'],
        ref_audio_strength = params['ref_audio_strength'],
        ref_audio_input = params['ref_audio_input'],
        save_path = params['audio_outputs_path']
    )
        
    return output_paths, input_params_json

### Initialize the results dataframe to File

In [ ]:
OUTPUT_DATAFRAME_COLUMNS = [
    'format',
    'lora_name_or_path',
    'lora_weight',
    'task',
    'prompt',
    'lyrics',
    'audio_duration',
    'infer_step',
    'guidance_scale',
    'scheduler_type',
    'cfg_type',
    'omega_scale',
    'guidance_interval',
    'guidance_interval_decay',
    'min_guidance_scale',
    'use_erg_tag',
    'use_erg_lyric',
    'use_erg_diffusion',
    'oss_steps',
    'timecosts',
    'manual_seeds',
    'actual_seeds',
    'retake_seeds',
    'retake_variance',
    'guidance_scale_text',
    'guidance_scale_lyric',
    'repaint_start',
    'repaint_end',
    'edit_n_min',
    'edit_n_max',
    'edit_n_avg',
    'src_audio_path',
    'edit_target_prompt',
    'edit_target_lyrics',
    'audio2audio_enable',
    'ref_audio_strength',
    'ref_audio_input',
    'audio_path',
    'use_erg',
    'complete',
    'compute_duration'
]

pd.DataFrame(data=[], columns=OUTPUT_DATAFRAME_COLUMNS).to_csv(
    grid_search_output_dataframe_filepath,
    mode='w', header=True, index=False
)

In [ ]:
# set(OUTPUT_DATAFRAME_COLUMNS).difference(set(all_params.keys()))
# all_param

# Run Grid Search!

In [ ]:
total_runs = grid_dataframe.shape[0]
for _n_runs, (_index, _row) in enumerate(grid_dataframe.iterrows()):
    print(f'Running inference {_n_runs} of {total_runs}...')
    input_params = _row.to_dict()

    if input_params.get('ref_audio_input') is not None:
        input_params['audio_duration'] = librosa.get_duration(path=input_params.get('ref_audio_input'))
        input_params['audio2audio_enable'] = True
    else:
        input_params['audio2audio_enable'] = False
        input_params['audio_duration'] = AUDIO_DURATION
        
    input_params['use_erg_tag'] = input_params.get('use_erg')
    input_params['use_erg_lyric'] = input_params.get('use_erg')
    input_params['use_erg_diffusion'] = input_params.get('use_erg')

    
    # Generate song
    t0 = time.process_time()
    output_path, input_params_json = generate_song(params=input_params)
    
    # update all_params
    all_params = merge_params(
        target=input_params,
        recorded=input_params_json
    )
    t1 = time.process_time()
    generation_time = t1-t0
    all_params['compute_duration'] = generation_time
    new_output_row_df = pd.json_normalize(all_params, max_level=0)
    new_output_row_df = new_output_row_df[OUTPUT_DATAFRAME_COLUMNS]
    new_output_row_df.to_csv(
        grid_search_output_dataframe_filepath,
        mode='a', header=False, index=False
    )

    # update input params if needing to restart later
    grid_dataframe.at[_index, 'complete'] = True
    grid_dataframe.to_csv(
        path_or_buf=grid_search_input_dataframe_filepath,
        mode='w', header=True, index=False
    )

In [ ]:
completed_df = pd.read_csv(
    grid_search_output_dataframe_filepath
)
completed_df

In [ ]:
# display(Audio(completed_df.iloc[0]['ref_audio_input']))
# for _audio_path in completed_df['audio_path']:
#     display(Audio(_audio_path))

# Misc / Abandoned

###### If running from command line to use gradio demo:
```
acestep \
python gui.py \
--torch_compile true \
--overlapped_decode true \
--lora_dir ./models/exp/ace_step_lora_rjwpersonal/ \
--port 7866

acestep \
--torch_compile true \
--cpu_offload true \
--overlapped_decode true \
--port 7866
```